<a href="https://colab.research.google.com/github/beifa/M5_Forecasting/blob/master/predict_clf_make_v2_tx1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gc
import pickle
import random
import numpy as np
import pandas as pd
import lightgbm as lgb
import psutil, warnings
from multiprocessing import Pool
PATH = '/kaggle/input/wmt-inc/'
PATH_MODEL = '/kaggle/input/model-wmt-clf/'

PATH_D1 = ''
PATH_D2 = ''
PATH_D3 = ''
PATH_D4 = '/kaggle/input/make-lags-pca/'
PATH_D5 = '/kaggle/input/make-trend-prob-sales/'
PATH_D6 = '/kaggle/input/make-lags-pca/'
features_saved = ''

SEED = 13
random.seed(SEED)
np.random.seed(SEED)

VER = 2000

In [ ]:
temp_df = pd.read_pickle(PATH_D1)
temp_df['pred'] = np.nan


def split_data(store, end = 0):
    
    remove_features = ['id','state_id','store_id', 'd','sales'] 
    
    encode_f = [
        'enc_dept_id_mean',
        'enc_item_id_mean',   
        'enc_item_id_state_id_mean',      
        'enc_item_id_store_id_mean'   
        ]#best var, 0.47034  

    df = pd.concat([
                pd.read_pickle(PATH_D1),
                pd.read_pickle(PATH_D1),
                pd.read_pickle(PATH_D1],
                axis =1)
    mask = (df.d >= end - 250) & (df.d < end)  
    df = df[mask].reset_index(drop=True)
    gc.collect()    
    
    lags = pd.read_pickle(PATH_D2)
    lags = lags[lags.index.isin(df.index)]
    df = pd.concat([df, lags], axis=1)
    del lags
    gc.collect()    

    encode = pd.read_pickle(PATH_D3)[encode_f]
    encode = encode[encode.index.isin(df.index)]
    df = pd.concat([df, encode], axis=1)
    del encode
    gc.collect()  
    
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d','sales']+features]   

    return df, features

In [ ]:
for store in STORES_IDS:
    print(store)       
    model_name = 'lgb_model_'+store+'_v'+str(VER)+'.bin' 
    estimator = pickle.load(open(PATH_MODEL + model_name, 'rb')) 
    for step in range(250, 2000, 250):
        print(step)
        df, features = split_data('False', end = step)
        mask = (temp_df.d >= step - 250) & (temp_df.d < step)  
        temp_df.loc[mask, f'pred_{store}'] = estimator.predict(df[features_saved]) 
        del df
        gc.collect()
    del estimator
    gc.collect()
    print('end')

TX_1
250
500
750
1000
1250
1500
1750
end


In [ ]:
temp_df.to_pickle('predict_steps_tx.pkl')